In [12]:
import numpy as np
import wntr
import networkx as nx

In [13]:
wn = wntr.network.WaterNetworkModel("rede-sub-a-alt (1).inp")

print("Number of junctions:", len(wn.junction_name_list))
print("Total nodes:", wn.num_nodes)
print("Total pipes:", wn.num_links)

Number of junctions: 44
Total nodes: 45
Total pipes: 60


In [14]:
# NetworkX graph
G = wn.get_graph()

# Fixed node order
node_list = list(G.nodes())
node_to_idx = {n: i for i, n in enumerate(node_list)}
num_nodes = len(node_list)

# Edge index (undirected)
edge_index = []
for u, v in G.edges():
    edge_index.append([node_to_idx[u], node_to_idx[v]])
    edge_index.append([node_to_idx[v], node_to_idx[u]])

edge_index = np.array(edge_index).T

print("edge_index shape:", edge_index.shape)

edge_index shape: (2, 120)


C:\Users\vedik\AppData\Local\Temp\ipykernel_31440\2147871450.py:2: DeprecationWarning: wntr.network.WaterNetworkModel.get_graph is deprecated, use wntr.network.WaterNetworkModel.to_graph instead
  G = wn.get_graph()


In [15]:
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

pressure_baseline = results.node["pressure"]
time_steps = pressure_baseline.index.tolist()

# baseline pressure at all times
P_baseline = pressure_baseline.copy()

print("Baseline simulation done")


Baseline simulation done


In [16]:
junction_list = wn.junction_name_list

# Leak magnitudes (you can adjust later)
leak_magnitudes = np.linspace(0.002, 0.04, 8)

print("Leak junctions:", len(junction_list))
print("Leak magnitudes:", leak_magnitudes)
print("Time steps:", time_steps)


Leak junctions: 44
Leak magnitudes: [0.002 0.007 0.013 0.018 0.024 0.029 0.035 0.04 ]
Time steps: [0, 600, 1200, 1800, 2400, 3000, 3600, 4200, 4800, 5400, 6000, 6600, 7200, 7800, 8400, 9000, 9600, 10200, 10800, 11400, 12000, 12600, 13200, 13800, 14400, 15000, 15600, 16200, 16800, 17400, 18000, 18600, 19200, 19800, 20400, 21000, 21600, 22200, 22800, 23400, 24000, 24600, 25200, 25800, 26400, 27000, 27600, 28200, 28800, 29400, 30000, 30600, 31200, 31800, 32400, 33000, 33600, 34200, 34800, 35400, 36000, 36600, 37200, 37800, 38400, 39000, 39600, 40200, 40800, 41400, 42000, 42600, 43200, 43800, 44400, 45000, 45600, 46200, 46800, 47400, 48000, 48600, 49200, 49800, 50400, 51000, 51600, 52200, 52800, 53400, 54000, 54600, 55200, 55800, 56400, 57000, 57600, 58200, 58800, 59400, 60000, 60600, 61200, 61800, 62400, 63000, 63600, 64200, 64800, 65400, 66000, 66600, 67200, 67800, 68400, 69000, 69600, 70200, 70800, 71400, 72000, 72600, 73200, 73800, 74400, 75000, 75600, 76200, 76800, 77400, 78000, 78600

In [17]:
X = []
y = []

for leak_junc in junction_list:
    for leak_size in leak_magnitudes:
        
        # reload clean network
        wn_leak = wntr.network.WaterNetworkModel("rede-sub-a-alt (1).inp")
        wn_leak.get_node(leak_junc).add_demand(leak_size, "leak")
        
        sim = wntr.sim.EpanetSimulator(wn_leak)
        results = sim.run_sim()
        pressure_leak = results.node["pressure"]
        
        for t in time_steps:
            delta_p = np.abs(
                P_baseline.loc[t] - pressure_leak.loc[t]
            )
            
            X.append(delta_p.loc[node_list].values)
            y.append(junction_list.index(leak_junc))


In [18]:
X = np.array(X)   # (N_samples, num_nodes)
y = np.array(y)   # (N_samples,)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Number of classes:", len(np.unique(y)))


X shape: (51040, 45)
y shape: (51040,)
Number of classes: 44


In [19]:
# Build adjacency list
adj = [[] for _ in range(num_nodes)]
for u, v in edge_index.T:
    adj[u].append(v)

X_aug = []

for sample in X:
    mean_dp = sample.mean()
    rel_dp = sample / (mean_dp + 1e-6)
    
    local_rel = []
    for i in range(num_nodes):
        neigh = adj[i]
        if len(neigh) == 0:
            local_rel.append(0.0)
        else:
            local_rel.append(
                sample[i] / (sample[neigh].mean() + 1e-6)
            )
    
    X_aug.append(
        np.stack([sample, rel_dp, local_rel], axis=1)
    )

X_aug = np.array(X_aug)

print("Augmented X shape:", X_aug.shape)

Augmented X shape: (51040, 45, 3)


In [20]:
np.save("X_rede-sub-a-alt.npy", X_aug)
np.save("y_rede-sub-a-alt.npy", y)
np.save("edge_index_rede-sub-a-alt.npy", edge_index)

print("rede-sub-a-alt dataset saved successfully")

rede-sub-a-alt dataset saved successfully


In [21]:
np.save("node_list_rede-sub-a-alt.npy", np.array(node_list))
np.save("junction_list_rede-sub-a-alt.npy", np.array(junction_list))